In [26]:
import warnings
import pandas as pd

In [27]:
from ftfy import fix_text
from rapidfuzz import process, fuzz

In [28]:
warnings.filterwarnings('ignore')

In [4]:
# Opción que deshabilita el limite de columnas y filas mostradas
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

---
## Carga data

In [5]:
# Path de los archivos procesados (formato parquet)
path_data = '../../data/clean'
df_business = pd.read_parquet(f'{path_data}/y_business_CLEAN.parquet')
df_review = pd.read_parquet(f'{path_data}/y_review_CLEAN.parquet')

---
---
## Análisis Exploratorio

### Setup

#### Dict target

In [6]:
# Diccionario de marcas de Darden y competidores
brands = {
    # Darden brands
    'darden': [
        "Olive Garden Italian Restaurant",
        "LongHorn Steakhouse",
        "Cheddar's Scratch Kitchen",
        "Yard House",
        "The Capital Grille",   ####
        "Seasons 52",
        "Bahama Breeze",
        "Eddie V's",  ####
        "Ruth's Chris Steak House"
    ],
    # Bloomin brands
    'bloomin': [
        "Outback Steakhouse",
        "Carrabba's Italian Grill",
        "Bonefish Grill",
        "Fleming's Prime Steakhouse & Wine Bar",
        "Aussie Grill",
        "Aussie Grill - Brandon"
    ],
    # Brinker brands
    'brinker': [
        "Chili's",
        "Chili's Grill & Bar",
        "Maggiano's Little Italy",
        "It's Just Wings"
    ],
    # Texas Roadhouse brands
    'texasroadhouse': [
        "Texas Roadhouse",
        "Bubba's 33",
        #"Jaggers"  # Fast food, excluir?   ####
    ]
}

### Filtro de brands

In [7]:
# Corrigir errores de decoding de texto, columna `name`
df_business['name'] = df_business['name'].apply(fix_text)

In [8]:
# Funcion que implementa busqueda 'fuzzy' en textos
def fuzzy_match(x, match_to, threshold=90):
    match, score, _ = process.extractOne(x, match_to, scorer=fuzz.WRatio)
    return match if score >= threshold else None

#### Darden

In [9]:
# Aplicar fuzzy matching para Darden
df_business['name_match'] = df_business['name'].apply(fuzzy_match, match_to=brands['darden'])
# Filtrar nulos (donde nulo representa no-match)
df_darden = df_business[df_business['name_match'].notnull()]
df_darden.drop('name_match', axis=1, inplace=True)
df_darden['name'].value_counts()

name
Olive Garden Italian Restaurant    10
LongHorn Steakhouse                 9
Cheddar's Scratch Kitchen           3
Ruth's Chris Steak House            3
Bahama Breeze                       2
Seasons 52                          2
Yard House                          1
Name: count, dtype: int64

In [32]:
df_darden['state'].value_counts()

state
PA    23
NJ     6
DE     1
Name: count, dtype: int64

#### Bloomin'

In [10]:
# Aplicar fuzzy matching para Bloomin
df_business['name_match'] = df_business['name'].apply(fuzzy_match, match_to=brands['bloomin'])
# Filtrar nulos (donde nulo representa no-match)
df_bloomin = df_business[df_business['name_match'].notnull()]
df_bloomin.drop('name_match', axis=1, inplace=True)
df_bloomin['name'].value_counts()

name
Outback Steakhouse                       13
Bonefish Grill                            7
Carrabba's Italian Grill                  4
Fleming's Prime Steakhouse & Wine Bar     3
Aussie Grill - Brandon                    1
Name: count, dtype: int64

In [31]:
df_bloomin['state'].value_counts()

state
PA    19
NJ     7
DE     2
Name: count, dtype: int64

#### Brinker

In [11]:
# Aplicar fuzzy matching para Brinker
df_business['name_match'] = df_business['name'].apply(fuzzy_match, match_to=brands['brinker'])
# Filtrar nulos (donde nulo representa no-match)
df_brinker = df_business[df_business['name_match'].notnull()]
df_brinker.drop('name_match', axis=1, inplace=True)
df_brinker['name'].value_counts()

name
Chili's                    20
Maggiano's Little Italy     3
It's Just Wings             1
Chili's Grill & Bar         1
Name: count, dtype: int64

In [30]:
df_brinker['state'].value_counts()

state
PA    19
DE     3
NJ     3
Name: count, dtype: int64

#### Texas Roadhouse

In [12]:
# Aplicar fuzzy matching para Texas Roadhouse
df_business['name_match'] = df_business['name'].apply(fuzzy_match, match_to=brands['texasroadhouse'])
# Filtrar nulos (donde nulo representa no-match)
df_texasroadhouse = df_business[df_business['name_match'].notnull()]
df_texasroadhouse.drop('name_match', axis=1, inplace=True)
df_texasroadhouse['name'].value_counts()

name
Texas Roadhouse    6
Bubba's 33         1
Name: count, dtype: int64

In [29]:
df_texasroadhouse['state'].value_counts()

state
PA    5
NJ    2
Name: count, dtype: int64

---
### Filtro de reviews por brand

In [13]:
business_cols = ['business_id', 'city', 'state', 'postal_code', 'coordinates']

In [14]:
df_review['day'] = df_review['date'].dt.day_name()

#### Darden

In [15]:
# Filtrar `df_review` por la col `business_id` en `df_darden`
filtro_id = df_darden['business_id']
# Filtrar los review
df_darden_review = df_review[df_review['business_id'].isin(filtro_id)]

# Corrigir errores de decoding de texto, columna `text`
df_darden_review['text'] = df_darden_review['text'].apply(fix_text)
df_darden_review.sample(1)

,name,review_id,user_id,business_id,stars,useful,funny,cool,text,date,day
456899,Seasons 52,rsvgVwnSfY0bYxOkHDfSRA,yVpl1JmMFe3V5Eb8WSckhw,mYMPepp0QIZRk_52pWzeoA,5.0,1,0,0,This place has s spot on for turkey day and an...,2016-11-24 23:05:19,Thursday


In [16]:
df_darden_review = pd.merge(df_darden_review, df_darden[business_cols], on='business_id', how='left')

# Fijar nueva columna `name` como la primer columna del df 
nom_col = df_darden_review.pop('name')
df_darden_review.insert(0, 'name', nom_col)

In [34]:
df_darden['state'].value_counts()

state
PA    23
NJ     6
DE     1
Name: count, dtype: int64

In [33]:
df_darden_review['state'].value_counts()

state
PA    5214
NJ    1075
DE      76
Name: count, dtype: int64

#### Bloomin'

In [17]:
# Filtrar `df_review` por la col `business_id` en `df_bloomin`
filtro_id = df_bloomin['business_id']
# Filtrar los review
df_bloomin_review = df_review[df_review['business_id'].isin(filtro_id)]

# Corrigir errores de decoding de texto, columna `text`
df_bloomin_review['text'] = df_bloomin_review['text'].apply(fix_text)
df_bloomin_review.sample(1)

,name,review_id,user_id,business_id,stars,useful,funny,cool,text,date,day
522811,Fleming’s Prime Steakhouse & Wine Bar,I25MEdroRWfmIb0fDJa20w,BM6B9ytwmEGfi9Cg0XUOdQ,6jr6xW05PcKutwr5lbVVAg,4.0,0,0,0,So it's time to think about a great meal out a...,2020-07-13 11:56:10,Monday


In [18]:
df_bloomin_review = pd.merge(df_bloomin_review, df_bloomin[business_cols], on='business_id', how='left')

# Fijar nueva columna `name` como la primer columna del df 
nom_col = df_bloomin_review.pop('name')
df_bloomin_review.insert(0, 'name', nom_col)

In [35]:
df_bloomin['state'].value_counts()

state
PA    19
NJ     7
DE     2
Name: count, dtype: int64

In [36]:
df_bloomin_review['state'].value_counts()

state
PA    3001
NJ    1090
DE     681
Name: count, dtype: int64

#### Brinker

In [19]:
# Filtrar `df_review` por la col `business_id` en `df_brinker`
filtro_id = df_brinker['business_id']
# Filtrar los review
df_brinker_review = df_review[df_review['business_id'].isin(filtro_id)]

# Corrigir errores de decoding de texto, columna `text`
df_brinker_review['text'] = df_brinker_review['text'].apply(fix_text)
df_brinker_review.sample()

,name,review_id,user_id,business_id,stars,useful,funny,cool,text,date,day
909577,Maggiano's Little Italy,oCuhmjTGyY5u4zpib6nZIg,0Nr7r5BJmYHbV_aDRhGnsQ,6r9FNW0-4aKqbPCX7UxTZg,5.0,0,0,0,"Disclaimer: For sentimental reasons, my wife a...",2017-07-12 23:12:06,Wednesday


In [20]:
df_brinker_review = pd.merge(df_brinker_review, df_brinker[business_cols], on='business_id', how='left')

# Fijar nueva columna `name` como la primer columna del df 
nom_col = df_brinker_review.pop('name')
df_brinker_review.insert(0, 'name', nom_col)

In [37]:
df_brinker['state'].value_counts()

state
PA    19
DE     3
NJ     3
Name: count, dtype: int64

In [38]:
df_brinker_review['state'].value_counts()

state
PA    2477
DE     523
NJ     209
Name: count, dtype: int64

#### Texas Roadhouse

In [21]:
# Filtrar `df_review` por la col `business_id` en `df_texasroadhouse`
filtro_id = df_texasroadhouse['business_id']
# Filtrar los review
df_texasroadhouse_review = df_review[df_review['business_id'].isin(filtro_id)]

# Corrigir errores de decoding de texto, columna `text`
df_texasroadhouse_review['text'] = df_texasroadhouse_review['text'].apply(fix_text)
df_texasroadhouse_review.sample()

,name,review_id,user_id,business_id,stars,useful,funny,cool,text,date,day
784665,Texas Roadhouse,SRBT_6dI6QHjWVsJYDEZPQ,8Z5kF5egvQPHgUXjOMlF6g,T9n_LqUUhC2b1ALhg57Y5Q,5.0,1,0,0,Walked in around 6pm for dinner. The hostess g...,2015-06-29 23:16:12,Monday


In [22]:
df_texasroadhouse_review = pd.merge(df_texasroadhouse_review, df_texasroadhouse[business_cols], on='business_id', how='left')

# Fijar nueva columna `name` como la primer columna del df 
nom_col = df_texasroadhouse_review.pop('name')
df_texasroadhouse_review.insert(0, 'name', nom_col)

In [39]:
df_texasroadhouse['state'].value_counts()

state
PA    5
NJ    2
Name: count, dtype: int64

In [40]:
df_texasroadhouse_review['state'].value_counts()

state
PA    938
NJ    434
Name: count, dtype: int64

#### Resumen y exportacion de filtros aplicados

In [23]:
print(
    "Cantidad de Reviews\n"
    f"Darden: {df_darden_review.shape[0]}\n"
    f"Bloomin': {df_bloomin_review.shape[0]}\n"
    f"Brinker: {df_brinker_review.shape[0]}\n"
    f"Texas RH: {df_texasroadhouse_review.shape[0]}"
)

Cantidad de Reviews
Darden: 6365
Bloomin': 4772
Brinker: 3209
Texas RH: 1372


---

In [24]:
dfs = [
    ['darden', df_darden_review],
    ['bloomin', df_bloomin_review],
    ['brinker', df_brinker_review],
    ['texasroadhouse', df_texasroadhouse_review]
]

drop_cols = ['useful','funny','cool']

for _, df in dfs:
    df.drop(columns=drop_cols, inplace=True)

In [25]:
#for nom, df in dfs:
#    df.to_parquet(f'reviews_{nom}.parquet')